## Práctico Final Planning 2020  

In [1]:
import argparse
import time
import msgpack
from enum import Enum, auto

import numpy as np

from planning_utils import a_star, heuristic, create_grid, read_global_home, prune_path
from udacidrone import Drone
from udacidrone.connection import MavlinkConnection
from udacidrone.messaging import MsgID
from udacidrone.frame_utils import global_to_local



In [2]:
class States(Enum):
    MANUAL = auto()
    ARMING = auto()
    TAKEOFF = auto()
    WAYPOINT = auto()
    LANDING = auto()
    DISARMING = auto()
    PLANNING = auto()


In [3]:
class MotionPlanning(Drone):

    def __init__(self, connection):
        super().__init__(connection)

        self.target_position = np.array([0.0, 0.0, 0.0])
        self.waypoints = []
        self.in_mission = True
        self.check_state = {}

        # initial state
        self.flight_state = States.MANUAL

        # register all your callbacks here
        self.register_callback(MsgID.LOCAL_POSITION, self.local_position_callback)
        self.register_callback(MsgID.LOCAL_VELOCITY, self.velocity_callback)
        self.register_callback(MsgID.STATE, self.state_callback)

    def local_position_callback(self):
        if self.flight_state == States.TAKEOFF:
            if -1.0 * self.local_position[2] > 0.95 * self.target_position[2]:
                self.waypoint_transition()
        elif self.flight_state == States.WAYPOINT:
            if np.linalg.norm(self.target_position[0:2] - self.local_position[0:2]) < 1.0:
                if len(self.waypoints) > 0:
                    self.waypoint_transition()
                else:
                    if np.linalg.norm(self.local_velocity[0:2]) < 1.0:
                        self.landing_transition()

    def velocity_callback(self):
        if self.flight_state == States.LANDING:
            if self.global_position[2] - self.global_home[2] < 0.1:
                if abs(self.local_position[2]) < 0.01:
                    self.disarming_transition()

    def state_callback(self):
        if self.in_mission:
            if self.flight_state == States.MANUAL:
                self.arming_transition()
            elif self.flight_state == States.ARMING:
                if self.armed:
                    self.plan_path()
            elif self.flight_state == States.PLANNING:
                self.takeoff_transition()
            elif self.flight_state == States.DISARMING:
                if ~self.armed & ~self.guided:
                    self.manual_transition()

    def arming_transition(self):
        self.flight_state = States.ARMING
        print("arming transition")
        self.arm()
        self.take_control()

    def takeoff_transition(self):
        self.flight_state = States.TAKEOFF
        print("takeoff transition")
        self.takeoff(self.target_position[2])

    def waypoint_transition(self):
        self.flight_state = States.WAYPOINT
        print("waypoint transition")
        self.target_position = self.waypoints.pop(0)
        print('target position', self.target_position)
        self.cmd_position(self.target_position[0], self.target_position[1], self.target_position[2], self.target_position[3])

    def landing_transition(self):
        self.flight_state = States.LANDING
        print("landing transition")
        self.land()

    def disarming_transition(self):
        self.flight_state = States.DISARMING
        print("disarm transition")
        self.disarm()
        self.release_control()

    def manual_transition(self):
        self.flight_state = States.MANUAL
        print("manual transition")
        self.stop()
        self.in_mission = False

    def send_waypoints(self):
        print("Sending waypoints to simulator ...")
        data = msgpack.dumps(self.waypoints)
        self.connection._master.write(data)

    def plan_path(self):
        self.flight_state = States.PLANNING
        print("Searching for a path ...")
        TARGET_ALTITUDE = 5
        SAFETY_DISTANCE = 6

        self.target_position[2] = TARGET_ALTITUDE

        # TODO: Leer lat0 lon0 de los datos del mapa
        lat0, lon0 = read_global_home('colliders.csv')
        
        # TODO: setear home position a (lon0, lat0, 0)
        self.set_home_position(lon0, lat0, 0)

        # TODO: Obtener la posicion global actual
        print('global position {}', format(self.global_position))
 
        # TODO: Convertir la posicion global a la posicion local using global_to_local()
        local_north, local_east, _ =global_to_local(self.global_position, self.global_home)
        
        print('global home {0}, position {1}, local position {2}'.format(self.global_home, self.global_position,
                                                                         self.local_position))
        inicio=time.time()
        print(inicio)
        
        # Leer el archivo colliders
        data = np.loadtxt('colliders.csv', delimiter=',', dtype='Float64', skiprows=2)
        
        # Crear un grid para cierta altitud 
        grid, north_offset, east_offset = create_grid(data, TARGET_ALTITUDE, SAFETY_DISTANCE)
        print("North offset = {0}, east offset = {1}".format(north_offset, east_offset))
        # Definir un punto de inicio en el centro del grid
        
        # TODO: Cambiar este grid start por la posicion real del drone segun lo que se leyo en el archivo
        grid_start_north = int(np.ceil(local_north - north_offset))
        grid_start_east = int(np.ceil(local_east - east_offset))
        grid_start = (grid_start_north, grid_start_east)
        print(grid_start)
        
        # El punto objetivo es arbitrario, se puede probar con varios diferentes
        # grid_goal = (-north_offset + 10, -east_offset + 10)
        # print(grid_goal)
        
        # TODO: Adaptarlo para usar longitus y latitus, convertirlo y usarlo como grid goal
        lat_goal, long_goal = 37.795757, -122.400162
        goal_global_position = np.array([long_goal, lat_goal, 0.0 ])
        
        local_north_goal, local_east_goal, _ = global_to_local(goal_global_position, self.global_home)
        
        grid_goal_north = int(np.ceil(local_north_goal - north_offset))
        grid_goal_east = int(np.ceil(local_east_goal - east_offset))
        grid_goal = (grid_goal_north, grid_goal_east)
        print(grid_goal)
        
        # Realizar la busqueda con A star
        # TODO: Le faltan los movimientos diagonales a la busqueda
        print('Local Start and Goal: ', grid_start, grid_goal)
        path, _ = a_star(grid, heuristic, grid_start, grid_goal)
        
        pruned_path=prune_path(path)
        
        final=time.time()
        print(final)
        
        demora=final-inicio
        print('TIEMPO EMPLEADO = ', demora)
       
        print(pruned_path)
        
        
        # TODO: optimizar el numero de waypoints
        # Convertir camino a Waypoints
        waypoints = [[p[0] + north_offset, p[1] + east_offset, TARGET_ALTITUDE, 0] for p in pruned_path]
        # Set self.waypoints
        print(waypoints)
        self.waypoints = waypoints
        # TODO: Enviar waypoints al simulador
        self.send_waypoints()

    def start(self):
        self.start_log("Logs", "NavLog.txt")

        print("starting connection")
        self.connection.start()

        # Only required if they do threaded
        # while self.in_mission:
        #    pass

        self.stop_log()


In [4]:
#if __name__ == "__main__":
parser = argparse.ArgumentParser()

In [5]:
parser.add_argument('--port', type=int, default=5760, help='Port number')

_StoreAction(option_strings=['--port'], dest='port', nargs=None, const=None, default=5760, type=<class 'int'>, choices=None, help='Port number', metavar=None)

In [6]:
parser.add_argument('--host', type=str, default='127.0.0.1', help="host address, i.e. '127.0.0.1'")

_StoreAction(option_strings=['--host'], dest='host', nargs=None, const=None, default='127.0.0.1', type=<class 'str'>, choices=None, help="host address, i.e. '127.0.0.1'", metavar=None)

In [7]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--port PORT] [--host HOST]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\sarto\AppData\Roaming\jupyter\runtime\kernel-914f5f18-e6fd-4052-b8ef-889aaf4e7ecf.json


SystemExit: 2

C:\Users\sarto\miniconda3\envs\fcnd\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
conn = MavlinkConnection('tcp:127.0.0.1:5760', timeout=600)

In [9]:
drone = MotionPlanning(conn)

Logs\TLog.txt


In [10]:
time.sleep(1)

In [11]:
drone.start()

Logs\NavLog.txt
starting connection
arming transition
Searching for a path ...
lat0 37.792480, lon0 -122.397450

global position {} [-122.3974501   37.7924798    0.243    ]
global home [-122.39745   37.79248    0.     ], position [-122.3974501   37.7924798    0.243    ], local position [-0.01523407 -0.01403669 -0.24330449]
1608300073.0310805
North offset = -316, east offset = -445
(316, 445)
(679, 204)
Local Start and Goal:  (316, 445) (679, 204)
Found a path.
1608300129.4725323
TIEMPO EMPLEADO =  56.44145178794861
[(316, 445), (339, 422), (344, 427), (396, 427), (460, 363), (453, 356), (453, 294), (454, 293), (482, 293), (483, 292), (484, 292), (485, 291), (486, 291), (487, 290), (488, 290), (489, 289), (491, 289), (492, 288), (493, 288), (494, 287), (495, 287), (496, 286), (497, 286), (498, 285), (499, 285), (500, 284), (502, 284), (503, 283), (504, 283), (505, 282), (506, 282), (507, 281), (508, 281), (509, 280), (511, 280), (512, 279), (513, 279), (514, 278), (515, 278), (516, 277)